# Bar Chart Race with Python
In this project, we will learn how to create an animated bar chart race in Python. This technique helps visualize changes in category values and rankings over time in a clear and compelling way.

[Bar Chart Race with Python — Aman Kharwal (2020)](https://amanxai.com/2020/08/14/bar-chart-race-with-python/)

<p align="center"><img src="PPP.png" width="600"></p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
import matplotlib.colors as mc
import colorsys
from random import randint
import re
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML
from itertools import cycle

## Data Read

In [2]:
df = pd.read_csv("gdp per capita.csv")

In [3]:
df = df.loc[df['Variable'] == 'GDP per capita in USA 2005 PPPs']

In [4]:
df = df[((df['Country'] != 'OECD - Total') &
         (df['Country'] != 'Non-OECD Economies') &
         (df['Country'] != 'World') &
         (df['Country'] != 'Euro area (15 countries)'))]

In [5]:
df = df[['Country', 'Time', 'Value']]

In [6]:
df = df.pivot(index='Country', columns='Time', values='Value')

In [7]:
df = df.reset_index()

## EDA - Exploratory Data Analysis

In [8]:
df.head()

Time,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
0,Australia,35398.590665,35772.924214,36574.111671,37001.057636,37490.826466,38110.903611,39081.674741,40110.730881,41119.059361,...,74769.700729,75848.994265,76951.820083,78077.978970,79226.109159,80395.166215,81585.566936,82797.128264,84029.468527,85282.463455
1,Austria,35273.436433,36215.035467,36388.028413,36461.675224,36905.129161,37602.374660,38506.765442,39377.618381,40173.350962,...,69251.109808,70354.509997,71473.060277,72604.572874,73749.122477,74905.246159,76071.478881,77248.830836,78435.623591,79631.399839
2,Belgium,33010.373492,33338.832038,33045.741412,32884.275841,33160.391539,33511.345193,33925.106580,34386.435771,34879.790211,...,60179.815805,61147.423813,62136.746899,63147.998053,64180.757337,65234.838504,66310.095858,67406.066911,68521.625516,69655.783971
3,Brazil,10092.706850,10278.468317,10294.364874,10439.872492,10537.292811,10682.048199,10905.714488,11139.049382,11364.933311,...,22928.534128,23402.068626,23869.303867,24326.510986,24770.148095,25197.115572,25605.114113,25992.919766,26360.934415,26710.594224
4,Canada,36287.105162,36815.296582,37068.128992,37443.910821,37994.480089,38652.738606,39075.794199,39458.625056,39855.877582,...,61638.419309,62500.160584,63377.574171,64269.378991,65174.021759,66090.702084,67019.421385,67959.704672,68911.106367,69873.476143


In [9]:
df.tail()

Time,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
35,Sweden,34209.769209,34928.522052,35067.549868,35305.363684,35999.124254,36846.276737,37689.252127,38547.182494,39426.021527,...,67777.467286,68506.121287,69235.037755,69965.240178,70702.137547,71447.234063,72209.567039,72993.411564,73801.228058,74630.533510
36,Switzerland,39237.661920,39498.173615,39491.493608,39860.293891,40252.454754,40886.275137,41488.334111,42100.053678,42734.647307,...,71012.105953,72043.085924,73091.142075,74153.492853,75232.756565,76327.612119,77438.293921,78564.089326,79705.352306,80857.705001
37,Turkey,12671.208365,13609.061057,13722.323662,14099.786221,14315.038203,14717.220312,15505.694116,16321.401830,17081.072684,...,39260.172488,39868.837611,40478.039811,41089.420918,41704.624248,42325.451694,42953.824278,43591.499092,44239.953289,44900.426057
38,United Kingdom,32894.536026,33044.269816,32910.906059,33231.881754,34065.183641,34741.647028,35412.586080,36138.536390,36923.154919,...,66903.166377,67806.056204,68717.782647,69638.854445,70569.020228,71507.904885,72455.341546,73410.435142,74371.487925,75336.488196
39,United States,43549.748960,43979.211475,44832.542753,45311.591727,46194.709032,47448.501361,48620.550768,49673.360335,50650.981527,...,81633.705427,82507.554592,83375.853540,84237.141585,85089.558511,85931.823014,86763.240742,87583.109151,88391.240992,89188.117560


In [10]:
df.sample(5)

Time,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
1,Austria,35273.436433,36215.035467,36388.028413,36461.675224,36905.129161,37602.374660,38506.765442,39377.618381,40173.350962,...,69251.109808,70354.509997,71473.060277,72604.572874,73749.122477,74905.246159,76071.478881,77248.830836,78435.623591,79631.399839
6,China (People's Republic of),6712.584079,7290.638697,7799.405258,8346.188800,8908.001766,9507.592483,10090.816949,10665.057916,11233.144030,...,33754.863339,34499.270215,35257.343130,36031.447889,36822.592078,37630.640700,38454.247133,39290.802493,40137.530049,40991.216569
30,Russia,15934.922475,16635.282944,17239.117606,17507.579449,17635.514901,18001.544649,18744.173060,19536.354269,20299.162260,...,41613.684205,42498.592547,43504.667313,44623.119051,45839.264847,47133.732211,48483.722623,49863.957120,51249.107119,52614.789091
28,Poland,17351.783007,18116.284337,18446.194627,18710.989241,19243.208212,19883.654412,20445.586770,20992.035095,21532.115582,...,37364.899189,37766.512650,38173.216344,38584.777595,39002.643251,39427.306398,39861.379272,40308.193994,40767.982859,41241.522261
8,Denmark,32393.522850,32624.677915,32393.115508,32410.769184,32764.538123,33234.199945,33724.745892,34191.226852,34645.594025,...,62931.486054,64117.890913,65310.779662,66508.238361,67710.061407,68914.329538,70122.911274,71332.589732,72547.683416,73765.491949


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 52 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  40 non-null     object 
 1   2010     40 non-null     float64
 2   2011     40 non-null     float64
 3   2012     40 non-null     float64
 4   2013     40 non-null     float64
 5   2014     40 non-null     float64
 6   2015     40 non-null     float64
 7   2016     40 non-null     float64
 8   2017     40 non-null     float64
 9   2018     40 non-null     float64
 10  2019     40 non-null     float64
 11  2020     40 non-null     float64
 12  2021     40 non-null     float64
 13  2022     40 non-null     float64
 14  2023     40 non-null     float64
 15  2024     40 non-null     float64
 16  2025     40 non-null     float64
 17  2026     40 non-null     float64
 18  2027     40 non-null     float64
 19  2028     40 non-null     float64
 20  2029     40 non-null     float64
 21  2030     40 non-nu

In [12]:
df.shape

(40, 52)

In [13]:
df.isnull().sum()

Time
Country    0
2010       0
2011       0
2012       0
2013       0
2014       0
2015       0
2016       0
2017       0
2018       0
2019       0
2020       0
2021       0
2022       0
2023       0
2024       0
2025       0
2026       0
2027       0
2028       0
2029       0
2030       0
2031       0
2032       0
2033       0
2034       0
2035       0
2036       0
2037       0
2038       0
2039       0
2040       0
2041       0
2042       0
2043       0
2044       0
2045       0
2046       0
2047       0
2048       0
2049       0
2050       0
2051       0
2052       0
2053       0
2054       0
2055       0
2056       0
2057       0
2058       0
2059       0
2060       0
dtype: int64

## Data Visualization

In [14]:

def is_year(col):
    s = str(col)
    return s.isdigit()

year_cols = [c for c in df.columns if c != 'Country' and is_year(c)]
year_cols = sorted(year_cols, key=lambda x: int(x))

df_long = df.melt(id_vars='Country', value_vars=year_cols, var_name='Year', value_name='Value').dropna()
df_long['Value'] = pd.to_numeric(df_long['Value'], errors='coerce')
df_long = df_long.dropna(subset=['Value'])
df_long['Year'] = df_long['Year'].astype(str)
year_cols = [str(c) for c in year_cols]

palette = list(plt.get_cmap('tab20').colors)
countries = sorted(df['Country'].unique().tolist())
color_cycle = cycle(palette)
country_color = {c: next(color_cycle) for c in countries}

def darken(rgb, amount=1.12):
    h, l, s = colorsys.rgb_to_hls(*rgb)
    r, g, b = colorsys.hls_to_rgb(h, max(0, 1 - amount*(1 - l)), s)
    return (r, g, b)

In [15]:
TITLE = "GDP per Capita (PPP)"
TOP_N = 8

def draw(year_label: str):
    ax.clear()
    d = (df_long[df_long['Year'].eq(year_label)]
         .nlargest(TOP_N, 'Value')
         .sort_values('Value', ascending=True))
    faces, edges = [], []
    for name in d['Country']:
        base = country_color[name]
        faces.append((base[0], base[1], base[2], 0.85))
        edges.append(darken(base, 1.12))
    ax.barh(d['Country'], d['Value'], color=faces, edgecolor=edges, linewidth=2)
    dx = (d['Value'].max() or 1) / 50
    for i, (val, name) in enumerate(zip(d['Value'], d['Country'])):
        ax.text(val + dx, i, f"{val:,.0f}", va="center", ha="left", fontsize=11)
    ax.set_title(f"{TITLE} — {year_label}", loc="right", fontsize=14)
    ax.set_xlabel("")
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.tick_params(axis='x', labelsize=10, colors='#666666')
    ax.grid(axis='x', alpha=0.3)
    ax.margins(0, 0.01)
    plt.box(False)


In [16]:
plt.close('all')

fig, ax = plt.subplots(figsize=(12, 8))
frames = year_cols + [year_cols[-1]] * 10
anim = FuncAnimation(fig, draw, frames=frames, interval=500, repeat=False)

display(HTML(anim.to_jshtml()))
plt.close(fig)

In [17]:
title_base = "GDP per Capita (PPP)"
gif_name = re.sub(r'[^A-Za-z0-9]+', '_', title_base).strip('_') + ".gif"

writer = PillowWriter(fps=20)
anim.save(gif_name, writer=writer, dpi=150)
print("Saved:", gif_name)

plt.close('all')

Saved: GDP_per_Capita_PPP.gif


## Conclusion
We built an animated bar chart race of GDP per capita (PPP): cleaned and reshaped the OECD data, drew frames, animated with FuncAnimation, and saved a GIF named from the chart title.
You learned how to prep time-series data, style clear horizontal bars, and export animations.